# Transform The Raw Data from Alheimsins/b5-johnson-120-ipip-neo-pi-r
Some of the questions may need some re-wording in order to activate the proper neurons in the models (hypothesis).
Add "I" at the beginning, remove extra information and also append a "choices" string.

We can perform 4 experiments to see if it makes a difference:
1) Questions begin with "I" - choices worded as "accurate"
1) Questions begin with "I" - choices worded as "agree"
1) Questions don't begin with "I" - choices worded as "accurate"
1) Questions don't begin with "I" - choices worded as "agree"

In [1]:
import json, sys
from typing import *

In [4]:
def preprocess_question(question: dict, prompt: str, prefix_i: bool = False) -> dict:
    """
    Pre-processes a single question into the desired format as specified in the optional arguments
    Args:
        question (dict): A single question from the JS script
        prefix_i (bool, optional): Pre-pends "I" to the beginning of the question and makes the original first word lowercase
        choices (str, optional): Choose between using the word "accurate" and "agree" for the choices string
        prompt(str, optional): The string on how to prompt the model to respond
    """
    
    # Remove question ID
    del question['id']

    # Check prefix boolean
    if prefix_i:
        question['text'] = question['text'][0].lower() + question['text'][1:]
        question['text'] = 'I ' + question['text'] + '\n'

    # Add choices - deprecated - no longer asking multiple choice question
    # Reason: models not trained specifically on multiple choice - easier to few-shot generation than multiple choice
    '''
    if choices == 'accurate':
        choice_string = '1) Very Inaccurate\n' + \
                        '2) Moderately Inaccurate\n' + \
                        '3) Neutral\n' + \
                        '4) Moderately Accurate\n' + \
                        '5) Very Accurate\n'
    elif choices == 'agree':
        choice_string = '1) Strongly Agree\n' + \
                        '2) Agree\n' + \
                        '3) Neutral\n' + \
                        '4) Disagree\n' + \
                        '5) Strongly Disagree\n'
    question['choice_string'] = choice_string
    '''
    # Construct a prefix string for open generation
    generation_prefix = f"{prompt}: You are happy.\n" + \
                        "I feel happy.\n\n" + \
                        f"{prompt}: You are sad.\n" + \
                        "I am sad.\n\n" + \
                        f"{prompt}: {question['text']}\n" + \
                        "I"

    # Translate keyed to reverse_score
    # This is because a minus key indicates that the question should correlate negatively with the trait/facet
    question['reverse_score'] = question['facet'] == 'plus'

    # Construct total string
    question['question'] = generation_prefix


In [5]:
prompt = 'Write out your thoughts on this statement'

with open('./data/raw_questions.json', 'r') as f:
    questions = json.load(f)
for question in questions:
    preprocess_question(question, prompt, prefix_i=True)

print(f"Fields in each question: {questions[0].keys()}")
print('A few examples')
for i in range(2):
    print(questions[i]['question'] + '\n')

dump = {}
dump['prompt'] = prompt
dump['questions'] = questions
# Dump to file
with open('./data/questions.json', 'w') as f:
    json.dump(dump, f)

Fields in each question: dict_keys(['text', 'keyed', 'domain', 'facet', 'reverse_score', 'question'])
A few examples
Write out your thoughts on this statement: You are happy.
I feel happy.

Write out your thoughts on this statement: You are sad.
I am sad.

Write out your thoughts on this statement: I worry about things

I

Write out your thoughts on this statement: You are happy.
I feel happy.

Write out your thoughts on this statement: You are sad.
I am sad.

Write out your thoughts on this statement: I make friends easily

I

